#Robot de limpieza reactivo

In [737]:
!pip3 install mesa

In [738]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 
from random import sample
import random

# Debido a que necesitamos un solo agente por celda elegimos `SingleGrid` que fuerza un solo objeto por celda.
from mesa.space import SingleGrid, MultiGrid

#IMPORTANTE CAMBIAR SingleGrid a MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Para poder desarollar el programa fue necesario la creacion de las siguientes clase:


*   Cleanner (Siendo la aspiradora que se muestra en el display)
*   Floor (Siendo la suciedad o la limpieza que se ve en el display)



In [739]:
totalM = 0
celda_sucia = 0

def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content,Cleanner):
              grid[x][y] = 2
            else:
                grid[x][y] = content.live
    return grid

class Cleanner(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.live = 2
    self.next_state = None


  def step(self):
    possible_steps = self.model.grid.get_neighborhood(
      self.pos,
      moore=True,
      include_center=False        
    )

    new_position = self.random.choice(possible_steps)
    self.model.grid.move_agent(self,new_position)

    global totalM 
    totalM+=1



class Floor(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    # self.live = np.random.choice([0,1])
    self.live = 1
    self.next_state = None

  def step(self):

    # this_cell = self.model.grid.get_cell_list_contents([self.pos])
    # dirt = [obj for obj in this_cell if isinstance(obj, Floor)]

    dirty = self.model.grid.get_cell_list_contents([self.pos])
    self.next_state =self.live 

    if len(dirty)>1:
      self.next_state = 0
      #Aqui inicia el conteo de las celdas limpias
      # global celda_sucia
      # celda_sucia += 1
      # if isinstance()
    else:
      if self.next_state == 0:
        self.next_state = 0
      else:
        self.next_state = 1 
                
      
  def advance(self):
    self.live = self.next_state
      
            
class GameLifeModel(Model):
    '''
    Define el modelo del juego de la vida.
    '''
    def __init__(self, width, height, initialDirt, vacuum):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = SimultaneousActivation(self)
        self.initialDirt = initialDirt # New ways to implement solution
        self.vacuum = vacuum

        # firs_coordinate = sample([x for x in range(0,width)],self.initialDirt)
        # second_coordinate = sample([x for x in range(0,height)],self.initialDirt)

        coord = []
        for i in range(width):
          for j in range(height):
            coord.append(str(i) + "," + str(j))
        random.shuffle(coord)

        # print (coord)

        bestCoord = np.random.choice(coord, initialDirt)
        
        for i in range(self.initialDirt):
          x = int(coord[i][0:1])
          y = int(coord[i][2:3])

          dirtyHarry = Floor((x,y),self)

          self.grid.place_agent(dirtyHarry, (x, y))
          self.schedule.add(dirtyHarry)
        
        for i in range(self.vacuum):
            a = Cleanner(i+initialDirt, self) #setting the spawn
            self.grid.place_agent(a, (1, 1))
            self.schedule.add(a)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
    
    def verification(self): #Verifica cuantas celdas sucias hay
      for celda in self.grid.coord_iter():
        content, x,y, = celda
        for i in content:
            if isinstance(i, Floor) and i.live == 1:
                global celda_sucia
                celda_sucia +=1

    # def limpio(self):
    #   counter = 0 # Definimos el counter del piso
    #   for celda in self.grid.coord_iter():
    #     content, x,y, = celda
    #     for i in content:
    #         if isinstance(i, Floor) and i.live == 1: #si esta sucio
    #           counter += 1
    #     if counter == 0:
    #       return True
    #     else:
    #       return False
        

                

    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.schedule.step()
        self.datacollector.collect(self)

# En esta apartado se definen los parametros con los que el programa inicializara 

In [740]:
# Definimos el tamaño del Grid
GRID_SIZE = 10
Grid_total = GRID_SIZE * GRID_SIZE
# Definimos el número de generaciones a correr
NUM_GENERATIONS = 300

#Porcentaje de suciedad del rango del 0 al 100 
module = 70
initialDirt = int((module*Grid_total) / 100)
#numero de bots 
bots = 11

#Definimos el tiempo maximo de ejecucion
tm = 0.06

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = GameLifeModel(GRID_SIZE, GRID_SIZE,initialDirt,bots)


# global celda_sucia
# while ( (time.time()-start_time) < tm and model.limpio() == False ):
#   model.step()

for i in range(NUM_GENERATIONS):
    model.step()
    # if not model.limpio():
    #   break

model.verification()

  


# Imprimimos el tiempo que le tomó correr al modelo.
# print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

In [741]:
all_grid = model.datacollector.get_model_vars_dataframe()

# all_grid = model.datacollector.get_model_vars_dataframe()
# print(type(all_grid))
# print(all_grid)

# Set de los valores gráficos

In [742]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
# En este campo se define el color de los agentes
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

# Display gráfico

In [743]:
anim

# Aqui podemos recopilar 




*   Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
*   Porcentaje de celdas limpias después del termino de la simulación.
*   Número de movimientos realizados por todos los agentes.



In [744]:

print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))
print("Porcentaje de celdas Limpias:", end =' ')
print(str(((GRID_SIZE*GRID_SIZE) - celda_sucia)* 100 / (GRID_SIZE*GRID_SIZE)) + " %")
# print(celda_sucia)
print("Total de Movientos", end =' ')
print(totalM)


  

Tiempo de ejecución: 0:00:19.586550
Porcentaje de celdas Limpias: 100.0 %
Total de Movientos 3300
